# Library to utilize Pandas ExcelWriter with ColInfo metadata

### Writing Pandas projects to an Excel workbook
***
This notebook contains Python functions useful for outputting an Excel, project workbook whose sheets are one or more Pandas DataFrames.  This is useful when a Python project creates several, related DataFrames whose data needs to be shared with a consulting client or other user.  Two use cases are possible --both involve calling the `WriteExcelWorkbook` function:
* with the `IsExcelSteps=False`, the function creates lists of column widths and Excel number formats and calls Pandas ExcelWriter that is internally called by `WriteExcelWorkbook`.  It attempts to source the formatting from `col_info.csv` in the `\libs\` sub-folder.
* with `IsExcelSteps=True` the function adds an Excelsteps sheet to the exported workbook --also sourcing the formatting info from `col_info.csv`.
Here is a background link on [Pandas ExcelWriter](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.ExcelWriter.html). [Here is a useful ExcelWriter code example](https://xlsxwriter.readthedocs.io/example_pandas_column_formats.html) from the documentation.  

J.D. Landgrebe,

March, 2020

In [1]:
import pandas as pd
import numpy as np

## XLWriter functions

In [2]:
#Loads new item into lists of DataFrames, sheet names.  Initializes column formats
def XLWriterPrep(lst_dfs, lst_shts, lst_fmts, lst_colwidths, df,sht):
    lst_fmts.append([])
    lst_colwidths.append([])
    for i in range(len(df.index.names) + len(df.columns)):
        lst_fmts[len(lst_fmts) - 1].append('')
        lst_colwidths[len(lst_colwidths) - 1].append(0)
    return lst_dfs, lst_shts, lst_fmts, lst_colwidths

# Write list of DataFrames to Excel workbook as separate worksheets
def XLWriter(wkbk, lst_dfs, lst_shts, lst_fmts, lst_colwidths):
    writer = pd.ExcelWriter(wkbk, engine='xlsxwriter')
    worksheet = []
    workbook = writer.book
    for i in range(len(lst_dfs)):
        lst_dfs[i].to_excel(writer, sheet_name=lst_shts[i])
        worksheet.append(writer.sheets[lst_shts[i]])
    
    #Add all uniqueformats to a dict
    dict_fmts = {}
    format = []
    k = 0
    for i in range(len(lst_fmts)):
        for j in range(len(lst_fmts[i])):
            curfmt = lst_fmts[i][j]
            if len(curfmt) > 0 and curfmt not in dict_fmts:
                dict_fmts[curfmt] = k #Save the index, k, as dictionary value for later
                format.append(workbook.add_format({'num_format': curfmt}))
                k += 1
    #Assign specified formats and column widths to each sheet
    for i in range(len(lst_shts)):
        
        #create pd.ExcelWriter object for each sheet
        worksheet = writer.sheets[lst_shts[i]]
        
        #Assign any specified column widths and number formats
        for j in range(1,len(lst_fmts[i])):
            colstr = XLColString(j + 1)
            colwidth = None
            fmt = None
            
            if lst_colwidths[i][j] > 0: colwidth = lst_colwidths[i][j]
            if len(lst_fmts[i][j]) > 0: fmt = lst_fmts[i][j]

            if fmt != None:
                worksheet.set_column(colstr, colwidth, format[dict_fmts[fmt]])
            else:
                worksheet.set_column(colstr, colwidth, None)
                
    #Write the workbook and return
    writer.save()
    return()

#Converts integer, icol, into Excel column range (Example: icol = 30 --> 'AD:AD')
def XLColString(icol):
    alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    letters =''
    while True:
        letter = ''
        Q = (icol - 1) // 26
        R = (icol - 1) % 26
        letter = alphabet[R]
        letters = letter + letters
        if Q == 0: break
        icol = Q
    return letters + ':' + letters

## Col_Info global variables and function

In [3]:
#Global Variables
NameCI = 'Name'
DescCI = 'Description'
UnitsCI = 'Units'
XLFormatCI = 'XLFormat'
XLWidthCI = 'XLWidth'
ParentCI = 'Parent'

path_ColInfo = 'libs/'

#ReadColInfo - reads the col_info DataFrame to a nested dictionary
#Column 1 of the col_info.csv contains column headings used as key for the other column info items (For example
#data column 'Revenue' has Description, XLFormat and XLWidth equal to 'Annual Revenue', '$0.00' and '11',
#respectively)
def ReadColInfoFromFile(PathCIFile):
    
    #Read the file, and Create the nested dictionary empty strucure
    df_CI = pd.read_csv(PathCIFile, index_col=NameCI)
    ColInfo = {DescCI: {}, UnitsCI: {}, XLFormatCI: {}, XLWidthCI: {}}

    #Read each col_info DataFrame row and translate to dictionary values
    for var_name, row in df_CI.iterrows():
        for dictCI in ColInfo:
            ColInfo[dictCI][var_name] = row[dictCI]
    return ColInfo

#RefreshColInfoToFile - refreshes col_info.csv based on dictionary contents; adds file rows as needed
def RefreshColInfoToFile(PathCIFile, ColInfo):
    
    #Read the col_info file to be updated
    df_CI = pd.read_csv(PathCIFile, index_col=NameCI)
    
    #Iterate over dictionaries in ColInfo and over variables (keys); update/append to df_CI
    for CI_dict in ColInfo:
        for var in CI_dict:
            df_CI.loc[var,CI_dict] = ColInfo[CI_dict][var]
    df_CI.to_csv(PathCIFile)
    return

#BuildXLWriterLists - Uses number format and column width ColInfo to build needed XLWriter lists
def BuildXLWriterLists(df, ColInfo):
    
    #Initialize lists with placeholder for index - Works for single index
    lst_fmts = ['0']
    lst_widths = [10]
    
    #Populate format and width for index; xxx Need to address case of multiindex
    if df.index.names[0] is None: 
        df.index.name = 'index'
    elif df.index.name in ColInfo:
        lst_fmts = [ColInfo[XLFormatCI][df.index.name]]
        lst_widths = [ColInfo[XLWidthCI][df.index.name]]

    #Iterate through df columns and add number format and column width to the lists
    for col in df.columns:
        if col in ColInfo[XLFormatCI] and col in ColInfo[XLWidthCI]:
            lst_fmts.append(ColInfo[XLFormatCI][col])
            lst_widths.append(ColInfo[XLWidthCI][col])
        else:
            lst_fmts.append('0')
            lst_widths.append(10)
    
    #Get rid of quotation marks that prevent formats from working with ExcelWriter
    lst_fmts = ListReplaceNaN(lst_fmts,'')
    lst_fmts = [s.replace('"', '') for s in lst_fmts]
    lst_widths = ListReplaceNaN(lst_widths,0)
    return lst_fmts, lst_widths

#Toggles val to blank string if val is nan values
def SetVal(val):
    retval = val
    if isinstance(val , float) and np.isnan(val): retval = ''
    return str(retval)
        
def CreateExcelStepsDF(lst_dfs, lst_XLshts, ColInfo):
    
    #Add needed ColInfo entries for ExcelSteps worksheet
    ColInfo[XLFormatCI]['Formula/List Name/Sort-by'] = '@'
    ColInfo[XLFormatCI]['Number Format'] = '@'

    dictC = {1:'Sheet', 2:'Column',3:'Step',4:'Comment',5:'Number Format',6:'Width'}
    df_ExcelSteps = pd.DataFrame(columns=['Sheet', 'Column', 'Step', 'Formula/List Name/Sort-by', 
                                          'After or End Column', 'Keep Formulas', 'Comment', 
                                          'Number Format', 'Width'])
    for dframe, sht in zip(lst_dfs, lst_XLshts):
        lst_dfCols = list(dframe.index.names) + dframe.columns.tolist()

        #Iterate through columns and append ExcelSteps recipe rows
        for i, col in enumerate(lst_dfCols):
            if not col in ColInfo[DescCI]: continue
            comment = SetVal(ColInfo[DescCI][col]) 
            ucomment = SetVal(ColInfo[UnitsCI][col])
            if len(ucomment) > 0: comment = comment + ' in ' + ucomment
                               
            row = {dictC[1]:sht, dictC[2]:col, dictC[3]:'Col_Format', dictC[4]:comment, 
                   dictC[5]: ColInfo[XLFormatCI][col],dictC[6]: ColInfo[XLWidthCI][col]}
            
            df_ExcelSteps = df_ExcelSteps.append(row, ignore_index=True)

        #Add Tbl_FreezeRow1 to end of recipe 
        df_ExcelSteps = df_ExcelSteps.append({dictC[1]:np.nan}, ignore_index=True)
        freeze = {dictC[1]:sht, dictC[3]:'Tbl_FreezeRow1'}
        df_ExcelSteps = df_ExcelSteps.append(freeze, ignore_index=True)

        #add blank row to create spacing in the recipe
        df_ExcelSteps = df_ExcelSteps.append({dictC[1]:np.nan}, ignore_index=True)
        
        #Name the ExcelSteps df's index
        df_ExcelSteps.index.name = 'row'
        
    lst_dfs.append(df_ExcelSteps)
    lst_XLshts.append('ExcelSteps')
    return lst_dfs, lst_XLshts, ColInfo

def WriteExcelWorkbook(lst_dfs, lst_shts, fName_xlsx, ColInfo, IsExcelSteps):
    lst_fmts = []
    lst_colwidths = []
    retval = fName_xlsx + ' Written Successfully'
    if len(lst_dfs) != len(lst_shts): return 'ERROR: Must be same number of dfs and shts'

    #Make local copy so ColInfo doesn't get modified
    Col_Info_l = ColInfo
    
    if IsExcelSteps:
        lst_dfs, lst_shts, Col_Info_l = CreateExcelStepsDF(lst_dfs, lst_shts, Col_Info_l)
        
    i=0
    for dframe, sht in zip(lst_dfs, lst_shts):
        XLWriterPrep(lst_dfs, lst_shts, lst_fmts, lst_colwidths, dframe, sht)        
        lst_fmts[i], lst_colwidths[i] = BuildXLWriterLists(dframe, ColInfo)
        i = i+1
        
    #Text format for ExcelSteps Formula and Number Format columns
    if IsExcelSteps:
        i = len(lst_dfs) - 1
        lst_fmts[i][4], lst_fmts[i][8] = '@', '@' 
    XLWriter(fName_xlsx, lst_dfs, lst_shts, lst_fmts, lst_colwidths)
    return retval

def ListReplaceNaN(lst, val_replace):
    for i, v in enumerate(lst):
        if v is np.nan: lst[i] = val_replace
    return lst

## XLWriter with ColInfo Demo
The library subroutines allow a single function call to`WriteExcelWorkbook()`: 
* The `XLWriterPrep` function manages lists of DataFrames and Excel Sheet Names to use for each DataFrame. Number Formats and Column Widths are managed as a list of lists (each element is a list of values for columns in each df).  
* When called, it appends an item to each of these lists --named items for DataFrame and Sheet Names; empty lists with the appropriate number of items for Number Formats and Column Widths
* Immediately after populating blank lists with `XLWriterPrep`, the `BuildXLWriterLists` function can be called to populate the Number Formats and Column Widths from ColInfo.
* When all DataFrames have been processed, the XLWriter function uses the lists to create an Excel Workbook

In [4]:
PathCIFile = 'col_info.csv'

#Import local, JDL utility modules
import sys
sys.path.append(sys.path[0] + '/libs/')
import excelwrite

### Create an example DataFrame and a second, summary DataFrame

In [5]:
IsLibraryCode = True

df = pd.DataFrame([['Company A','Product A',27.46],
                   ['Company B','Product B',18.33],
                   ['Company C','Product C',14.0],
                   ['Company A','Product A',19.27], 
                   ['Company C','Product B',94.17],
                   ['Company B','Product B',18.13],
                   ['Company B','Product B',15.05],
                   ['Company C','Product B',19.25],
                   ['Company A','Product B',27.6]], 
                  columns=['Company','Product','Revenue'])
df['NewCol'] = 1
df.index.name = 'row_order'
df

,Company,Product,Revenue,NewCol
row_order,,,,
0,Company A,Product A,27.46,1
1,Company B,Product B,18.33,1
2,Company C,Product C,14.00,1
3,Company A,Product A,19.27,1
4,Company C,Product B,94.17,1
5,Company B,Product B,18.13,1
6,Company B,Product B,15.05,1
7,Company C,Product B,19.25,1
8,Company A,Product B,27.60,1


In [6]:
df_summ = df.groupby(['Company','Product'], as_index=False).sum()
df_summ.index.name = 'row_order'
df_summ

,Company,Product,Revenue,NewCol
row_order,,,,
0,Company A,Product A,46.73,2
1,Company A,Product B,27.60,1
2,Company B,Product B,51.51,3
3,Company C,Product B,113.42,2
4,Company C,Product C,14.00,1


### Export workbook using ExcelWriter

In [8]:
print(path_ColInfo + PathCIFile, '\n')
IsExcelSteps = True

if IsLibraryCode: 
    ColInfo = excelwrite.ReadColInfoFromFile(path_ColInfo + PathCIFile)
    msg = excelwrite.WriteExcelWorkbook([df, df_summ], ['Sales Data', 'Sales Summary'], 'All.xlsx', ColInfo, IsExcelSteps)
else: 
    ColInfo = ReadColInfoFromFile(path_ColInfo + PathCIFile)
    msg = WriteExcelWorkbook([df, df_summ], ['Sales Data', 'Sales Summary'], 'All.xlsx', ColInfo, IsExcelSteps)    

print(msg,'\n')
for dict in ColInfo:
    print(ColInfo[dict])

libs/col_info.csv 

All.xlsx Written Successfully 

{'row_order': 'row (index)', 'Company': 'Company Name', 'Product': 'Product Name', 'Revenue': 'Annual Revenue'}
{'row_order': nan, 'Company': nan, 'Product': nan, 'Revenue': '$US'}
{'row_order': '0', 'Company': '@', 'Product': '@', 'Revenue': '$#,##0.00', 'Formula/List Name/Sort-by': '@', 'Number Format': '@'}
{'row_order': 10, 'Company': 12, 'Product': 15, 'Revenue': 17}
